### Objetivos del laboratorio
- Comparar rendimiento y características entre un dataset en Parquet y el mismo dataset convertido a Delta Lake.
- Aplicar particionamiento para optimizar consultas.
- Usar Time Travel para consultar versiones anteriores de la tabla después de modificaciones.

### 1. Lectura inicial del dataset en formato CSV

/Volumes/sesion4/default/source/csv/

In [0]:
from pyspark.sql.types import StructType, StringType, DoubleType, DateType

polizas_schema = (
    StructType()
    .add("id_poliza", StringType())
    .add("fecha_emision", DateType())
    .add("tipo_seguro", StringType())
    .add("prima_anual", DoubleType())
    .add("estado_poliza", StringType())
    .add("region", StringType())
    .add("nombre_cliente", StringType())
)

In [0]:
df_polizas_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .schema(polizas_schema)
    .load("/Volumes/sesion4/default/source/csv/")
)

df_polizas_csv.count()

### 2. Escribir a formato parquet

In [0]:
df_polizas_csv.write.format("parquet").mode("overwrite").save("/Volumes/sesion4/default/source/parquet")

### 3. Leer formato parquet

In [0]:
df_polizas_parquet = spark.read.format("parquet").load("/Volumes/sesion4/default/source/parquet")

df_polizas_parquet.count()

### 4. Escribir formato delta con particion

In [0]:
df_polizas_parquet.write.format("delta").mode("overwrite").partitionBy("region").saveAsTable("sesion4.default.polizas_con_particion")

### 5. Escribir formato delta sin particion

In [0]:
df_polizas_parquet.write.format("delta").mode("overwrite").saveAsTable("sesion4.default.polizas_sin_particion")

In [0]:
%sql
select region, count(1)
from sesion4.default.polizas_sin_particion
group by all

In [0]:
%sql
select *
from sesion4.default.polizas_sin_particion
WHERE region = 'Norte'

In [0]:
%sql
select *
from sesion4.default.polizas_con_particion
WHERE region = 'Norte'

In [0]:
%sql
DESCRIBE DETAIL sesion4.default.polizas_sin_particion

### 6. Time Travel

In [0]:
%sql
SELECT estado_poliza, count(1)
FROM sesion4.default.polizas_con_particion
WHERE tipo_seguro = 'Auto'
GROUP BY all


In [0]:
%sql
UPDATE sesion4.default.polizas_con_particion
SET estado_poliza = 'Cancelada'
WHERE tipo_seguro = 'Auto'

In [0]:
%sql
SELECT estado_poliza, count(1)
FROM sesion4.default.polizas_con_particion
WHERE tipo_seguro = 'Auto'
GROUP BY all


In [0]:
%sql
DESCRIBE HISTORY sesion4.default.polizas_con_particion

In [0]:
%sql
RESTORE TABLE sesion4.default.polizas_con_particion VERSION AS OF 0

In [0]:
%sql
SELECT estado_poliza, count(1)
FROM sesion4.default.polizas_con_particion
WHERE tipo_seguro = 'Auto'
GROUP BY all


### Cuestionario

1. ¿Qué formato es columnar por defecto?
- CSV
- JSON
- Parquet
- TXT

2. ¿Cuál NO es una ventaja de Delta Lake?
- ACID
- Time travel
- Z-Ordering
- Mayor tamaño de archivo por diseño

3. DESCRIBE HISTORY sirve para…
- Ver el lineage en Fabric
- Ver versiones/operaciones Delta
- Comprimir la tabla
- Quitar archivos huérfanos

4. Para consultar una versión pasada sin restaurar:
- SELECT … FROM tabla VERSION AS OF 3
- RESTORE TABLE tabla TO VERSION AS OF 3
- OPTIMIZE ZORDER BY (3)
- VACUUM

5. ¿Cuál es verdadera sobre Delta + esquema?
- No permite cambiar esquema 
- Permite evolución controlada
- Requiere siempre MERGE 
- Solo funciona en JSON

6. ¿Qué hace el transaction log?
- Guarda solo el esquema 
- Orquesta jobs 
- Registra commits/estado de archivos 
- Desactiva ACID

7. Convertir Parquet a Delta sin reescritura lógica es…
- CREATE TABLE … USING DELTA AS SELECT …
- CONVERT TO DELTA delta.
- CONVERT TO DELTA parquet.ruta
- RESTORE TABLE

8. ¿Qué comando muestra metadatos/estadísticas de una tabla Delta?
- DESCRIBE DETAIL 
- SHOW PARTITIONS 
- MSCK REPAIR 
- SHOW CREATE TABLE